<a href="https://colab.research.google.com/github/abarrie2/cs598-dlh-project/blob/main/DL4H_Team_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grading rubric

## Draft (20 Points)

Jupyter Notebook (both .PDF and .ipynb files)

You need to use the report template and fill out the following sections, each of which we will score based on the clarity and appropriateness of your writing (percentage of total grade for each component shown). All the information must be in the Jupyter notebook.

- Introduction (2)
  - A clear, high-level description of what the original paper is about and what is the contribution of it
- Scope of reproducibility (2)
- Methodology (8)
  - Data
    - Data descriptions
    - Implementation code
  - Model
    - Model descriptions
    - Implementation code
  - Training
    - Computational requirements
    - Implementation code
  - Evaluation
    - Metrics descriptions
    - Implementation code
- Results (8)
  - Results
  - Analyses
  - Plans

## Final Descriptive Notebook Report (55 Points)

Jupyter Notebook (both in .PDF and .ipynb format)

- Introduction (5):
    - A clear, high-level description of what the original paper is about and what is the contribution of it
- Scope of reproducibility (5)
- Methodology (15)
  - Environment
    - Python version
    - Dependencies/packages needed
  - Data
    - Data download instruction
    - Data descriptions with helpful charts and visualizations
    - Preprocessing code + command
  - Model
    - Citation to the original paper
    - Link to the original paper’s repo (if applicable)
    - Model descriptions
    - Implementation code
    - Pretrained model (if applicable)
  - Training
    - Hyperparams
      - Report at least 3 types of hyperparameters such as learning rate, batch size, hidden size, dropout
    - Computational requirements
      - Report at least 3 types of requirements such as type of hardware, average runtime for each epoch, total number of trials, GPU hrs used, # training epochs
      - Training code
  - Evaluation
    - Metrics descriptions
    - Evaluation code
- Results (15)
  - Table of results (no need to include additional experiments, but main reproducibility result should be included)
  - All claims should be supported by experiment results
  - Discuss with respect to the hypothesis and results from the original paper
  - Experiments beyond the original paper
    - Credits for each experiment depend on how hard it is to run the experiments. Each experiment should include results and a discussion
    - Ablation Study.
- Discussion (10)
  - Implications of the experimental results, whether the original paper was reproducible, and if it wasn’t, what factors made it irreproducible
  - “What was easy”
  - “What was difficult”
  - Recommendations to the original authors or others who work in this area for improving reproducibility
- Public GitHub Repo (5)
  - Publish your code in a public repository on GitHub and attach the URL in the notebook.
  - Make sure your code is documented properly. 
    - A README.md file describing the exact steps to run your code is required. 
    - Check [ML Code Completeness Checklist](https://github.com/paperswithcode/releasing-research-code)
    - Check [Best Practices for Reproducibility](https://www.cs.mcgill.ca/~ksinha4/practices_for_reproducibility/)

---

# FAQ and Attentions
* Copy and move this template to your Google Drive. Name your notebook by your team ID (upper-left corner). Don't eidt this original file.
* This template covers most questions we want to ask about your reproduction experiment. You don't need to exactly follow the template, however, you should address the questions. Please feel free to customize your report accordingly.
* any report must have run-able codes and necessary annotations (in text and code comments).
* The notebook is like a demo and only uses small-size data (a subset of original data or processed data), the entire runtime of the notebook including data reading, data process, model training, printing, figure plotting, etc,
must be within 8 min, otherwise, you may get penalty on the grade.
  * If the raw dataset is too large to be loaded  you can select a subset of data and pre-process the data, then, upload the subset or processed data to Google Drive and load them in this notebook.
  * If the whole training is too long to run, you can only set the number of training epoch to a small number, e.g., 3, just show that the training is runable.
  * For results model validation, you can train the model outside this notebook in advance, then, load pretrained model and use it for validation (display the figures, print the metrics).
* The post-process is important! For post-process of the results,please use plots/figures. The code to summarize results and plot figures may be tedious, however, it won't be waste of time since these figures can be used for presentation. While plotting in code, the figures should have titles or captions if necessary (e.g., title your figure with "Figure 1. xxxx")
* There is not page limit to your notebook report, you can also use separate notebooks for the report, just make sure your grader can access and run/test them.
* If you use outside resources, please refer them (in any formats). Include the links to the resources if necessary.

# Mount Notebook to Google Drive
Upload the data, pretrianed model, figures, etc to your Google Drive, then mount this notebook to Google Drive. After that, you can access the resources freely.

Instruction: https://colab.research.google.com/notebooks/io.ipynb

Example: https://colab.research.google.com/drive/1srw_HFWQ2SMgmWIawucXfusGzrj1_U0q

Video: https://www.youtube.com/watch?v=zc8g8lGcwQU

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Introduction
This is an introduction to your report, you should edit this text/mardown section to compose. In this text/markdown, you should introduce:

*   Background of the problem
  * what type of problem: disease/readmission/mortality prediction,  feature engineeing, data processing, etc
  * what is the importance/meaning of solving the problem
  * what is the difficulty of the problem
  * the state of the art methods and effectiveness.
*   Paper explanation
  * what did the paper propose
  * what is the innovations of the method
  * how well the proposed method work (in its own metrics)
  * what is the contribution to the reasearch regime (referring the Background above, how important the paper is to the problem).


In [ ]:
# code comment is used as inline annotations for your coding

# Scope of Reproducibility:

List hypotheses from the paper you will test and the corresponding experiments you will run.


1.   Hypothesis 1: xxxxxxx
2.   Hypothesis 2: xxxxxxx

You can insert images in this notebook text, [see this link](https://stackoverflow.com/questions/50670920/how-to-insert-an-inline-image-in-google-colaboratory-from-google-drive) and example below:

![sample_image.png](https://drive.google.com/uc?export=view&id=1g2efvsRJDxTxKz-OY3loMhihrEUdBxbc)



You can also use code to display images, see the code below.

The images must be saved in Google Drive first.


In [ ]:
# no code is required for this section
'''
if you want to use an image outside this notebook for explanaition,
you can upload it to your google drive and show it with OpenCV or matplotlib
'''
# mount this notebook to your google drive
drive.mount('/content/gdrive')

# define dirs to workspace and data
img_dir = '/content/gdrive/My Drive/Colab Notebooks/<path-to-your-image>'

import cv2
img = cv2.imread(img_dir)
cv2.imshow("Title", img)


# Methodology

This methodology is the core of your project. It consists of run-able codes with necessary annotations to show the expeiment you executed for testing the hypotheses.

The methodology at least contains two subsections **data** and **model** in your experiment.

### Create environment

Create `conda` environment for the project using the `environment.yml` file:

```bash
conda env create --prefix .envs/dlh-team24 -f environment.yml
```

Activate the environment with:
```bash
conda activate .envs/dlh-team24
```

In [ ]:
# Import packages
import numpy as np
import pandas as pd
import vitaldb
#from google.colab import drive


##  Data
Data includes raw data (MIMIC III tables), descriptive statistics (our homework questions), and data processing (feature engineering).
  * Source of the data: where the data is collected from; if data is synthetic or self-generated, explain how. If possible, please provide a link to the raw datasets.
  * Statistics: include basic descriptive statistics of the dataset like size, cross validation split, label distribution, etc.
  * Data process: how do you munipulate the data, e.g., change the class labels, split the dataset to train/valid/test, refining the dataset.
  * Illustration: printing results, plotting figures for illustration.
  * You can upload your raw dataset to Google Drive and mount this Colab to the same directory. If your raw dataset is too large, you can upload the processed dataset and have a code to load the processed dataset.

In [ ]:
# Read track and case information from vitaldb
df_cases = pd.read_csv("https://api.vitaldb.net/cases")  # clinical information
df_trks = pd.read_csv("https://api.vitaldb.net/trks")  # track list

print(f'{len(df_cases)} total cases')

# As in the paper, select cases which meet the following criteria:
# The inclusion criteria were as follows:
# (1) adults (age>18)
# (2) administered general anaesthesia
# (3) undergone non-cardiac surgery. 
# The exclusion criteria were as follows:
# (1) any missing monitoring for ABP, ECG, and EEG waveforms
# (2) cases containing false events or non-events due to poor signal quality (checked in second stage of data preprocessing)
inclusion_1 = set(df_cases.loc[df_cases['age'] > 18, 'caseid']) # adult
print(f'{len(df_cases)-len(inclusion_1)} cases excluded due to age criteria')
inclusion_2 = set(df_cases.loc[df_cases['ane_type'] == 'General', 'caseid']) # general anesthesia
print(f'{len(df_cases)-len(inclusion_2)} cases excluded due to anesthesia criteria')
inclusion_3 = set(df_cases.loc[~df_cases['opname'].str.contains("cardiac", case=False),'caseid']) & set(df_cases.loc[~df_cases['opname'].str.contains("aneurysmal", case=False),'caseid']) # non-cardiac surgery
print(f'{len(df_cases)-len(inclusion_3)} cases excluded due to non-cardiac surgery criteria')
exclusion_1 = set(df_trks.loc[df_trks['tname'] == 'SNUADC/ART', 'caseid']) & set(df_trks.loc[df_trks['tname'] == 'SNUADC/ECG_II', 'caseid']) & set(df_trks.loc[df_trks['tname'] == 'BIS/EEG1_WAV', 'caseid']) # ABP, ECG, EEG waveforms
print(f'{len(df_cases)-len(exclusion_1)} cases excluded due to missing waveform data')

caseids = list(
    inclusion_1 & inclusion_2 & inclusion_3 & exclusion_1
)
print(f'{len(caseids)} cases matching all criteria')

In [ ]:
# Load data tracks
# TODO: Load ABP, ECG, EEG waveforms for each caseid

In [ ]:
# Exclude cases where ABP j signal quality (jSQI) < 0.8
# TODO: Implement jSQI function
# TODO: Filter cases with jSQI < 0.8

In [ ]:
# Generate hypotensive events
# Hypotensive events are defined as a 1-minute interval with sustained ABP of less than 65 mmHg
# Note: Hypotensive events should be at least 20 minutes apart to minimize potential residual effects from previous events
# TODO: Implement hypotension event generation function
# TODO: Generate hypotension events

# Generate hypotension non-events
# To sample non-events, 30-minute segments where the ABP was above 75 mmHG were selected, and then
# three one-minute samples of each waveform were obtained from the middle of the segment
# TODO: Implement hypotension non-event generation function
# TODO: Generate hypotension non-events

In [ ]:
# Preprocess data tracks

# ABP waveforms are used without further pre-processing
# ECG waveforms are band-pass filtered between 1 and 40 Hz, and Z-score normalized
# EEG waveforms are band-pass filtered between 0.5 and 40 Hz

In [ ]:
# Split data into training, validation, and test sets
# Use 6:1:3 ratio and prevent samples from a single case from being split across different sets
# Note: number of samples at each time point is not the same, because the first event can occur before the 3/5/10/15 minute mark

In [ ]:
# dir and function to load raw data
raw_data_dir = '/content/gdrive/My Drive/Colab Notebooks/<path-to-raw-data>'

def load_raw_data(raw_data_dir):
  # implement this function to load raw data to dataframe/numpy array/tensor
  return None

raw_data = load_raw_data(raw_data_dir)

# calculate statistics
def calculate_stats(raw_data):
  # implement this function to calculate the statistics
  # it is encouraged to print out the results
  return None

# process raw data
def process_data(raw_data):
    # implement this function to process the data as you need
  return None

processed_data = process_data(raw_data)

''' you can load the processed data directly
processed_data_dir = '/content/gdrive/My Drive/Colab Notebooks/<path-to-raw-data>'
def load_processed_data(raw_data_dir):
  pass

'''

##   Model
The model includes the model definitation which usually is a class, model training, and other necessary parts.
  * Model architecture: layer number/size/type, activation function, etc
  * Training objectives: loss function, optimizer, weight of each loss term, etc
  * Others: whether the model is pretrained, Monte Carlo simulation for uncertainty analysis, etc
  * The code of model should have classes of the model, functions of model training, model validation, etc.
  * If your model training is done outside of this notebook, please upload the trained model here and develop a function to load and test it.

In [ ]:
class my_model():
  # use this class to define your model
  pass

model = my_model()
loss_func = None
optimizer = None

def train_model_one_iter(model, loss_func, optimizer):
  pass

num_epoch = 10
# model training loop: it is better to print the training/validation losses during the training
for i in range(num_epoch):
  train_model_one_iter(model, loss_func, optimizer)
  train_loss, valid_loss = None, None
  print("Train Loss: %.2f, Validation Loss: %.2f" % (train_loss, valid_loss))


# Results
In this section, you should finish training your model training or loading your trained model. That is a great experiment! You should share the results with others with necessary metrics and figures.

Please test and report results for all experiments that you run with:

*   specific numbers (accuracy, AUC, RMSE, etc)
*   figures (loss shrinkage, outputs from GAN, annotation or label of sample pictures, etc)


In [ ]:
# metrics to evaluate my model

# plot figures to better show the results

# it is better to save the numbers and figures for your presentation.

## Model comparison

In [ ]:
# compare you model with others
# you don't need to re-run all other experiments, instead, you can directly refer the metrics/numbers in the paper

# Discussion

In this section,you should discuss your work and make future plan. The discussion should address the following questions:
  * Make assessment that the paper is reproducible or not.
  * Explain why it is not reproducible if your results are kind negative.
  * Describe “What was easy” and “What was difficult” during the reproduction.
  * Make suggestions to the author or other reproducers on how to improve the reproducibility.
  * What will you do in next phase.



In [ ]:
# no code is required for this section
'''
if you want to use an image outside this notebook for explanaition,
you can read and plot it here like the Scope of Reproducibility
'''

# References

1.   Sun, J, [paper title], [journal title], [year], [volume]:[issue], doi: [doi link to paper]



# Feel free to add new sections